# Finch usage

Finch is a WPS server for climate indicators, but also has a few utilities to facilitate data handling. To get started, first instantiate the client. 

In [1]:
import os
from birdy import WPSClient

pavics_url = 'https://pavics.ouranos.ca/twitcher/ows/proxy/finch/wps'
url = os.environ.get('FINCH_WPS_URL', pavics_url)
verify_ssl = True if 'DISABLE_VERIFY_SSL' not in os.environ else False
wps = WPSClient(url, verify=verify_ssl)

The list of available processes can be displayed using the help function, as well as details about individual processes. 

In [2]:
wps?

In [3]:
wps.frost_days?

To actually compute an indicator, we need to specify the path to the netCDF file used as input for the calculation of the indicator. To compute `frost_days`, we need a time series of daily minimum temperature. Here we'll use a small test file. Note that here we're using an OPeNDAP link, but it could also be an url to a netCDF file, or the path to a local file on disk. We then simply call the indicator. The response is an object that can poll the server to inquire about the status of the process. This object can use two modes: 
 - synchronous: it will wait for the server's response before returning; or 
 - asynchronous: it will return immediately, but without the actual output from the process.
 
Here, since we're applying the process on a small test file, we're using the default synchronous mode. For long computations, use the asynchronous mode to avoid time-out errors. The asynchronous mode is activated by setting the `progress` attribute of the WPS client to True. 

In [4]:
tasmin = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/testdata/flyingpigeon/cmip3/tasmin.sresa2.miub_echo_g.run1.atm.da.nc"
resp = wps.frost_days(tasmin)

In [5]:
print(resp.status)
out = resp.get()
print(out)

ProcessSucceeded
frost_daysResponse(
    output_netcdf='http://https://pavics.ouranos.ca/wpsoutputs/e8afbb04-42d3-11ea-9531-0242ac12000b/frost-days_SRES-A2-experiment_20460101-20650101.nc',
    output_log='https://pavics.ouranos.ca/wpsoutputs/e8afbb04-42d3-11ea-9531-0242ac12000b/log.txt',
    ref='https://pavics.ouranos.ca/wpsoutputs/e8afbb04-42d3-11ea-9531-0242ac12000b/input.meta4'
)


The `get` method returns a `NamedTuple` object with all the WPS outputs, either as references to files or actual content. To copy the file to the local disk, you can use the `getOutput` method. 

In [6]:
resp.getOutput('/tmp/out.nc')

In [7]:
import xarray as xr
xr.open_dataset('/tmp/out.nc')

<xarray.Dataset>
Dimensions:     (lat: 6, lon: 7, time: 20)
Coordinates:
    height      float64 ...
  * time        (time) object 2046-01-01 00:00:00 ... 2065-01-01 00:00:00
  * lat         (lat) float64 42.68 46.39 50.1 53.81 57.52 61.23
  * lon         (lon) float64 281.2 285.0 288.8 292.5 296.2 300.0 303.8
Data variables:
    frost_days  (time, lat, lon) timedelta64[ns] ...
Attributes:
    comment:                  Spinup: restart files from end of experiment 20...
    title:                    MIUB  model output prepared for IPCC Fourth Ass...
    cmor_version:             0.96
    institution:              Canadian Centre for Climate Services (CCCS)
    source:                   ECHO-G(1999): atmosphere: ECHAM4 (T30L19) with ...
    contact:                  Canadian Centre for Climate Services
    references:               ECHAM4: E. Roeckner et al., 1996, The atmospher...
    experiment_id:            SRES A2 experiment
    realization:              1
    directory:                /ipcc/sresa2/atm/da/
    table_id:                 Table A2 (17 November 2004)
    calendar:                 360_day
    project_id:               IPCC Fourth Assessment
    Conventions:              CF-1.0
    id:                       pcmdi.ipcc4.miub_echo_g.sresa2.run1.atm.da
    history:                  Mon Aug  1 11:43:58 2011: ncks -4 -L 7 -d lat,4...
    NCO:                      4.0.9
    climateindex_package_id:  https://github.com/Ouranosinc/xclim
    product:                  derived climate index
    institute_id:             CCCS

The birdy client offers a quicker way to download and open the files automatically using `asobj=True`, as long as the file format is known to birdy. 

In [8]:
ds, log, metalink = resp.get(asobj=True)

In [9]:
ds

<xarray.Dataset>
Dimensions:     (lat: 6, lon: 7, time: 20)
Coordinates:
    height      float64 ...
  * time        (time) object 2046-01-01 00:00:00 ... 2065-01-01 00:00:00
  * lat         (lat) float64 42.68 46.39 50.1 53.81 57.52 61.23
  * lon         (lon) float64 281.2 285.0 288.8 292.5 296.2 300.0 303.8
Data variables:
    frost_days  (time, lat, lon) timedelta64[ns] ...
Attributes:
    comment:                  Spinup: restart files from end of experiment 20...
    title:                    MIUB  model output prepared for IPCC Fourth Ass...
    cmor_version:             0.96
    institution:              Canadian Centre for Climate Services (CCCS)
    source:                   ECHO-G(1999): atmosphere: ECHAM4 (T30L19) with ...
    contact:                  Canadian Centre for Climate Services
    references:               ECHAM4: E. Roeckner et al., 1996, The atmospher...
    experiment_id:            SRES A2 experiment
    realization:              1
    directory:                /ipcc/sresa2/atm/da/
    table_id:                 Table A2 (17 November 2004)
    calendar:                 360_day
    project_id:               IPCC Fourth Assessment
    Conventions:              CF-1.0
    id:                       pcmdi.ipcc4.miub_echo_g.sresa2.run1.atm.da
    history:                  Mon Aug  1 11:43:58 2011: ncks -4 -L 7 -d lat,4...
    NCO:                      4.0.9
    climateindex_package_id:  https://github.com/Ouranosinc/xclim
    product:                  derived climate index
    institute_id:             CCCS

In [10]:
# NBVAL_IGNORE_OUTPUT
print(log)

Computing the output netcdf
Processing file 0 of 1
[               ] | 0% Done
[###############] | 100% Done
Processing finished successfully

